## UBP x Paymaya project

### Aim: 


#### Background: 


*import libraries*

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import networkx as nx
import bokeh

In [2]:
import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_rows = 50
pd.options.display.max_columns = 50

#### level 2

*import data*

levels 
<br> Level1 (one edge):
UBP to OtherUBP/ UBP to paymaya

Level 2 (two edges):
UBP to UBP to other NonUBP (gcash, paymaya, and nonubp) accounts (if applicable)

Level 3 (three edges or more):
UBP to UBP to UBP to other NonUBP (gcash, paymaya, and nonubp) accounts (if applicable)

##### create level 2 dataset

In [7]:
txn_df = pd.read_csv('/home/cdsw/Data/Final_Full_Data.csv')

In [8]:
txn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960933 entries, 0 to 960932
Data columns (total 29 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     960933 non-null  int64  
 1   Unnamed: 0.1                   960933 non-null  int64  
 2   Unnamed: 0_x                   960933 non-null  int64  
 3   mcif                           960933 non-null  int64  
 4   alertid                        737146 non-null  object 
 5   tran_date                      960933 non-null  object 
 6   currency                       960933 non-null  object 
 7   source_name                    960933 non-null  object 
 8   target_name                    830862 non-null  object 
 9   TargetBank                     960933 non-null  object 
 10  txn_amount                     960933 non-null  float64
 11  target_acct                    960933 non-null  object 
 12  from_acct                     

mcif - mastercif of customer
<br> alertid - specific alert
<br> source_name - name
<br> tenure - tenure of customer 
<br> report_dt - report date/transaction date
<br> sameday_C_ATM_amt - total amt of inflow ATM txn within same day before report date 
<br> sameday_D_ft_amt - total amount of outflow fund transfer transactions within same day before report date
<br> sameday_D_ft_count - number of outflow fund transfer transactions within same day before report date 
<br> 1day_C_ft_amt - total amount of inflow fund transfer transactions within 1 day before report date
<br> 2mon_C_others_count - number of inflow other transactions within 2 months before report date
<br> sameday_Android_UBlogin_count - number of UB online app logins from android devices within same day before report date
<br> response - '0' STR alert is not promoted to case ; '1' STR alert promoted to case 

In [9]:
txn_df.shape

(960933, 29)

In [10]:
txn_df['TargetBank'].unique()

array(['UBP', 'NonUBP', 'paymaya', 'Gcash'], dtype=object)

In [11]:
txn_df['currency'].value_counts()

PHP    960855
USD        78
Name: currency, dtype: int64

**filter for ubp to ubp txn**

In [12]:
ubp_1 = txn_df[txn_df['TargetBank']=='UBP'][['tran_date','source_name','target_name','TargetBank','txn_amount','response']].dropna(subset = ['target_name'])


In [13]:
ubp_1 = ubp_1[ubp_1['source_name']!=ubp_1['target_name']]
ubp_1.head(10)

,tran_date,source_name,target_name,TargetBank,txn_amount,response
0,2020-10-26,aleli joanne martin curameng,raniemar reyes curameng,UBP,300.00,0
247,2021-04-14,personal savings,james isoy,UBP,7051.92,1
254,2021-04-14,personal savings,james isoy,UBP,1.00,1
297,2021-04-14,personal savings,clarissa oligo,UBP,9058.07,1
377,2021-02-03,personal savings,grant,UBP,100000.00,1
379,2021-02-03,personal savings,lynub,UBP,5000.00,1
380,2021-02-03,personal savings,lynub,UBP,32000.00,1
381,2021-02-03,personal savings,rocklyn,UBP,80000.00,1
382,2021-02-03,personal savings,lynub,UBP,6150.00,1
384,2021-02-03,personal savings,lynub,UBP,13253.00,1


**filter for ubp to non-ubp txn**

In [14]:
ubp_2 = txn_df[txn_df['TargetBank']!='UBP'][['tran_date','source_name','target_name','TargetBank','txn_amount','response']].dropna(subset = ['target_name'])

In [15]:
ubp_2 = ubp_2[ubp_2['source_name']!=ubp_2['target_name']]
ubp_2.head(10)

,tran_date,source_name,target_name,TargetBank,txn_amount,response
1,2020-12-21,personal savings,allen rose mallari,NonUBP,13016.29,1
2,2020-12-21,personal savings,cedie atienza,paymaya,21878.35,1
3,2020-12-21,personal savings,ellen grace r layco,NonUBP,15699.71,1
4,2020-12-21,personal savings,lee ann cahilig,NonUBP,13406.61,1
5,2020-12-21,personal savings,nathaniel d patropes,NonUBP,9072.32,1
6,2020-12-21,personal savings,cedie atienza,Gcash,21878.35,1
7,2020-12-21,personal savings,casey tumang,Gcash,9035.92,1
8,2020-12-21,personal savings,venn jimiel foronda tadeo,Gcash,16068.01,1
9,2020-12-21,personal savings,ralph martin b beltran,NonUBP,7346.74,1
10,2020-12-21,personal savings,jonathan abawag,NonUBP,8025.86,1


In [16]:
# merge both such that ubp to ubp to non-ubp
big_df = pd.merge(ubp_1, ubp_2, left_on=['target_name'], right_on=['source_name'])

In [17]:
big_df.shape

(21556545, 12)

In [18]:
big_df.head(10)

,tran_date_x,source_name_x,target_name_x,TargetBank_x,txn_amount_x,response_x,tran_date_y,source_name_y,target_name_y,TargetBank_y,txn_amount_y,response_y
0,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-03-29,digipay,oliven,Gcash,1000.0,1
1,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-03-29,digipay,janrens,Gcash,1094.0,1
2,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-03-29,digipay,wesley,Gcash,1400.0,1
3,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-03-29,digipay,gernie,Gcash,1000.0,1
4,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-03-29,digipay,kamille,Gcash,2300.0,1
5,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-03-29,digipay,benedict,NonUBP,3000.0,1
6,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-03-29,digipay,charmae,NonUBP,6000.0,1
7,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-03-29,digipay,royland,Gcash,500.0,1
8,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-03-29,digipay,jonel,Gcash,2000.0,1
9,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-03-29,digipay,arturo,Gcash,1233.0,1


In [19]:
big_df['day_diff'] = (pd.to_datetime(big_df['tran_date_y']) - pd.to_datetime(big_df['tran_date_x'])).dt.days

In [20]:
# transfers have to be done within 1 day. 
level_2 = big_df[(big_df['day_diff']<=1) & (big_df['day_diff']>0)]
level_2

,tran_date_x,source_name_x,target_name_x,TargetBank_x,txn_amount_x,response_x,tran_date_y,source_name_y,target_name_y,TargetBank_y,txn_amount_y,response_y,day_diff
88527,2021-03-29,personal savings,digipay,UBP,30000.0,1,2021-03-30,digipay,irvin,NonUBP,35000.0,1,1
88528,2021-03-29,personal savings,digipay,UBP,30000.0,1,2021-03-30,digipay,jonel,Gcash,2000.0,1,1
88529,2021-03-29,personal savings,digipay,UBP,30000.0,1,2021-03-30,digipay,jayvie,paymaya,100.0,1,1
88530,2021-03-29,personal savings,digipay,UBP,30000.0,1,2021-03-30,digipay,mark,Gcash,2000.0,1,1
88531,2021-03-29,personal savings,digipay,UBP,30000.0,1,2021-03-30,digipay,joseph,Gcash,2000.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21556117,2021-03-03,jeaneaiko alison calderon,madeleine denise verano,UBP,100000.0,0,2021-03-04,madeleine denise verano,stevien,NonUBP,3900.0,1,1
21556118,2021-03-03,jeaneaiko alison calderon,madeleine denise verano,UBP,100000.0,0,2021-03-04,madeleine denise verano,shayne,NonUBP,50000.0,1,1
21556119,2021-03-03,jeaneaiko alison calderon,madeleine denise verano,UBP,100000.0,0,2021-03-04,madeleine denise verano,shayne,NonUBP,30000.0,1,1
21556120,2021-03-03,jeaneaiko alison calderon,madeleine denise verano,UBP,100000.0,0,2021-03-04,madeleine denise verano,jaaron,NonUBP,14000.0,1,1


In [21]:
# transfers have to be done within 1 day and the first transactions
level_2_first = big_df[(big_df['day_diff']<=1) & (big_df['day_diff']>0)].groupby(by=['tran_date_x','source_name_x','target_name_x','TargetBank_x','txn_amount_x','response_x']).first().reset_index()
level_2_first 

,tran_date_x,source_name_x,target_name_x,TargetBank_x,txn_amount_x,response_x,tran_date_y,source_name_y,target_name_y,TargetBank_y,txn_amount_y,response_y,day_diff
0,2020-07-15,marietta aguiguin,irish joy de vega,UBP,500.0,0,2020-07-16,irish joy de vega,tingap,Gcash,1810.0,0,1
1,2020-07-20,aegel rose de castro,mylene villaflores,UBP,1500.0,0,2020-07-21,mylene villaflores,patrick flores,NonUBP,3300.0,0,1
2,2020-07-20,aegel rose de castro,wynona tracey ralutin,UBP,15000.0,0,2020-07-21,wynona tracey ralutin,yllysa torreno,Gcash,6800.0,1,1
3,2020-07-20,mc rone repomanta,marky cymon repomanta,UBP,16100.0,0,2020-07-21,marky cymon repomanta,kharla cathrine fulgencio,NonUBP,22770.0,0,1
4,2020-07-20,mc rone repomanta,marky cymon repomanta,UBP,67800.0,0,2020-07-21,marky cymon repomanta,kharla cathrine fulgencio,NonUBP,22770.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6467,2021-05-28,shelly sharmaine rb kaamino,christian angelo delos santos,UBP,245000.0,1,2021-05-29,christian angelo delos santos,kimberly flores,Gcash,6500.0,1,1
6468,2021-05-28,shelly sharmaine rb kaamino,jessa navarro,UBP,150000.0,1,2021-05-29,jessa navarro,maria lourdes l,NonUBP,1800.0,1,1
6469,2021-05-28,shelo ampoloquio,jahn steve evangelista,UBP,200000.0,1,2021-05-29,jahn steve evangelista,burton delico,NonUBP,5000.0,1,1
6470,2021-05-28,sweeny lumbang,maria gutierrez,UBP,4000.0,1,2021-05-29,maria gutierrez,niel valderrama,NonUBP,2000.0,1,1


##### export level 2 data

In [22]:
level_2.to_csv('/home/cdsw/Data/sy_level_2.csv')

In [23]:
level_2_first.to_csv('/home/cdsw/Data/sy_level_2_first.csv')

#### level 3

##### create level 3 dataset

In [28]:
# merge both such that ubp to ubp to ubp
tmp2 = pd.merge(ubp_1, ubp_1, left_on=['target_name'], right_on=['source_name'])
tmp2.head(10)

,tran_date_x,source_name_x,target_name_x,TargetBank_x,txn_amount_x,response_x,tran_date_y,source_name_y,target_name_y,TargetBank_y,txn_amount_y,response_y
0,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2020-11-09,digipay,jayvie pe,UBP,24500.0,1
1,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2020-11-09,digipay,mark cyril yu,UBP,1000.0,1
2,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2020-11-09,digipay,boss dee,UBP,30000.0,1
3,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-05-17,digipay,raymart,UBP,22000.0,1
4,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-05-17,digipay,oliver,UBP,37880.0,1
5,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-05-17,digipay,raymart batalla,UBP,10000.0,1
6,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2020-12-21,digipay,rodel,UBP,3000.0,1
7,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2020-12-21,digipay,diana mae manzano,UBP,1500.0,1
8,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2020-12-21,digipay,roshiel luriz,UBP,5000.0,1
9,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2020-12-21,digipay,rodel,UBP,1500.0,1


In [29]:
tmp2.shape

(5909465, 12)

In [30]:
tmp2['day_diff'] = (pd.to_datetime(tmp2['tran_date_y']) - pd.to_datetime(tmp2['tran_date_x'])).dt.days

In [31]:
# transfers have to be done within 1 day. 
ubp_1_1 = tmp2[(tmp2['day_diff']<=1) & (tmp2['day_diff']>0)]
ubp_1_1

,tran_date_x,source_name_x,target_name_x,TargetBank_x,txn_amount_x,response_x,tran_date_y,source_name_y,target_name_y,TargetBank_y,txn_amount_y,response_y,day_diff
25,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-05-18,digipay,edsel,UBP,24916.0,1,1
26,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-05-18,digipay,honey alcantara,UBP,43015.0,1,1
27,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-05-18,digipay,mark,UBP,1238.0,1,1
28,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-05-18,digipay,adam,UBP,11839.0,1,1
29,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-05-18,digipay,ralphy,UBP,11864.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5908985,2021-05-28,rovile jean banares,mikael jasper golez,UBP,148072.0,1,2021-05-29,mikael jasper golez,jerald eracho araza,UBP,5000.0,1,1
5909244,2021-03-03,jeaneaiko alison calderon,madeleine denise verano,UBP,100000.0,0,2021-03-04,madeleine denise verano,angelique mae biares,UBP,180000.0,1,1
5909245,2021-03-03,jeaneaiko alison calderon,madeleine denise verano,UBP,100000.0,0,2021-03-04,madeleine denise verano,vanessa arguelles,UBP,120000.0,1,1
5909246,2021-03-03,jeaneaiko alison calderon,madeleine denise verano,UBP,100000.0,0,2021-03-04,madeleine denise verano,angelique mae biares,UBP,230000.0,1,1


In [33]:
ubp_1_1.shape

(75887, 13)

In [34]:
# ubp_2 - ubp to non-ubp
ubp_2.head()

,tran_date,source_name,target_name,TargetBank,txn_amount,response
1,2020-12-21,personal savings,allen rose mallari,NonUBP,13016.29,1
2,2020-12-21,personal savings,cedie atienza,paymaya,21878.35,1
3,2020-12-21,personal savings,ellen grace r layco,NonUBP,15699.71,1
4,2020-12-21,personal savings,lee ann cahilig,NonUBP,13406.61,1
5,2020-12-21,personal savings,nathaniel d patropes,NonUBP,9072.32,1


In [35]:
# ubp_1_1 - ubp to ubp to ubp
# ubp_2 - ubp to non-ubp
# merge both such that ubp to ubp to ubp to non-ubp
tmp3 = pd.merge(ubp_1_1, ubp_2, left_on=['target_name_y'], right_on=['source_name'])
tmp3.head(10)

,tran_date_x,source_name_x,target_name_x,TargetBank_x,txn_amount_x,response_x,tran_date_y,source_name_y,target_name_y,TargetBank_y,txn_amount_y,response_y,day_diff,tran_date,source_name,target_name,TargetBank,txn_amount,response
0,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-05-18,digipay,adam,UBP,11839.0,1,1,2020-11-09,adam,gilbert,NonUBP,37200.0,1
1,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-05-18,digipay,adam,UBP,11839.0,1,1,2020-11-09,adam,joyce mendoza,NonUBP,5488.0,1
2,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-05-18,digipay,adam,UBP,11839.0,1,1,2020-11-09,adam,doraysa,Gcash,13705.0,1
3,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-05-18,digipay,adam,UBP,11839.0,1,1,2020-11-09,adam,priyah,Gcash,5488.0,1
4,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-05-18,digipay,adam,UBP,11839.0,1,1,2020-11-09,adam,la arni,Gcash,16449.0,1
5,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-05-18,digipay,adam,UBP,11839.0,1,1,2020-11-09,adam,emi rald,Gcash,5473.0,1
6,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-05-18,digipay,adam,UBP,11839.0,1,1,2020-11-09,adam,diana,Gcash,2435.0,1
7,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-05-18,digipay,adam,UBP,11839.0,1,1,2020-11-09,adam,arlene,Gcash,488.0,1
8,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-05-18,digipay,adam,UBP,11839.0,1,1,2020-11-09,adam,flerie,NonUBP,4900.0,1
9,2021-05-17,alvie ponayo,digipay,UBP,5000.0,1,2021-05-18,digipay,adam,UBP,11839.0,1,1,2020-11-09,adam,mherlyn,Gcash,5473.0,1


In [36]:
tmp3.shape

(11403246, 19)

_x - ubp to ubp1
<br> _y - ubp1 to ubp2
<br> no suffix - ubp2 to non-ubp

In [37]:
tmp3['day_diff'] = (pd.to_datetime(tmp3['tran_date']) - pd.to_datetime(tmp3['tran_date_y'])).dt.days

In [38]:
# transfers between ubp2 to non-ubp
# transfers have to be done within 1 day, and first transactions
level_3 = tmp3[(tmp3['day_diff']<=1) & (tmp3['day_diff']>0)]
level_3

,tran_date_x,source_name_x,target_name_x,TargetBank_x,txn_amount_x,response_x,tran_date_y,source_name_y,target_name_y,TargetBank_y,txn_amount_y,response_y,day_diff,tran_date,source_name,target_name,TargetBank,txn_amount,response
142427,2021-03-04,john ildefonso villanueva,jennifer iracta,UBP,40000.0,1,2021-03-05,jennifer iracta,john ildefonso villanueva,UBP,135158.0,1,1,2021-03-06,john ildefonso villanueva,jover s,Gcash,2000.0,1
142428,2021-03-04,john ildefonso villanueva,jennifer iracta,UBP,40000.0,1,2021-03-05,jennifer iracta,john ildefonso villanueva,UBP,135158.0,1,1,2021-03-06,john ildefonso villanueva,kenneth caguitla,NonUBP,1000.0,1
142429,2021-03-04,john ildefonso villanueva,jennifer iracta,UBP,40000.0,1,2021-03-05,jennifer iracta,john ildefonso villanueva,UBP,135158.0,1,1,2021-03-06,john ildefonso villanueva,gerald de vera,Gcash,2000.0,1
142430,2021-03-04,john ildefonso villanueva,jennifer iracta,UBP,40000.0,1,2021-03-05,jennifer iracta,john ildefonso villanueva,UBP,135158.0,1,1,2021-03-06,john ildefonso villanueva,romero s,Gcash,900.0,1
142431,2021-03-04,john ildefonso villanueva,jennifer iracta,UBP,40000.0,1,2021-03-05,jennifer iracta,john ildefonso villanueva,UBP,135158.0,1,1,2021-03-06,john ildefonso villanueva,anthony s,Gcash,1000.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11383843,2021-04-29,claryza fello,russ jermain pelle,UBP,50000.0,1,2021-04-30,russ jermain pelle,claryza fello,UBP,140000.0,1,1,2021-05-01,claryza fello,joanna joy petelo,NonUBP,50000.0,1
11383844,2021-04-29,claryza fello,russ jermain pelle,UBP,50000.0,1,2021-04-30,russ jermain pelle,claryza fello,UBP,140000.0,1,1,2021-05-01,claryza fello,marilyn ralcones,NonUBP,1000.0,1
11383845,2021-04-29,claryza fello,russ jermain pelle,UBP,50000.0,1,2021-04-30,russ jermain pelle,claryza fello,UBP,140000.0,1,1,2021-05-01,claryza fello,gretchen v penales,NonUBP,15000.0,1
11383846,2021-04-29,claryza fello,russ jermain pelle,UBP,50000.0,1,2021-04-30,russ jermain pelle,claryza fello,UBP,140000.0,1,1,2021-05-01,claryza fello,gretchen v penales,NonUBP,30000.0,1


In [39]:
# transfers between ubp2 to non-ubp
# transfers have to be done within 2 days, and first transactions
level_3_first = tmp3[(tmp3['day_diff']<=1) & (tmp3['day_diff']>0)].groupby(by=['tran_date_y','source_name_y','target_name_y','TargetBank_y','txn_amount_y','response_y']).first().reset_index()
level_3_first

,tran_date_y,source_name_y,target_name_y,TargetBank_y,txn_amount_y,response_y,tran_date_x,source_name_x,target_name_x,TargetBank_x,txn_amount_x,response_x,day_diff,tran_date,source_name,target_name,TargetBank,txn_amount,response
0,2020-10-06,jacenth joi montano,julie anne guinto,UBP,60000.0,0,2020-10-05,julie anne guinto,jacenth joi montano,UBP,400000.0,0,1,2020-10-07,julie anne guinto,richmon jaysam atienza,NonUBP,50000.0,0
1,2020-10-20,gabriel catulay,ivan terado,UBP,12000.0,1,2020-10-19,ivan terado,gabriel catulay,UBP,32000.0,1,1,2020-10-21,ivan terado,geng i,Gcash,2500.0,1
2,2020-10-20,gabriel catulay,ivan terado,UBP,40000.0,1,2020-10-19,ivan terado,gabriel catulay,UBP,32000.0,1,1,2020-10-21,ivan terado,geng i,Gcash,2500.0,1
3,2020-10-20,gabriel catulay,ivan terado,UBP,100000.0,1,2020-10-19,ivan terado,gabriel catulay,UBP,32000.0,1,1,2020-10-21,ivan terado,geng i,Gcash,2500.0,1
4,2020-10-20,gabriel catulay,ivan terado,UBP,150000.0,1,2020-10-19,ivan terado,gabriel catulay,UBP,32000.0,1,1,2020-10-21,ivan terado,geng i,Gcash,2500.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,2021-05-28,merly pagayon,maricel ramos,UBP,70352.0,1,2021-05-27,nina gallinera,merly pagayon,UBP,6318.0,1,1,2021-05-29,maricel ramos,isidro labarnes,Gcash,11000.0,1
920,2021-05-28,mylene baldon,dennis damian,UBP,4000.0,1,2021-05-27,erdinel cruz,mylene baldon,UBP,6000.0,1,1,2021-05-29,dennis damian,rogelio villangca,NonUBP,50000.0,1
921,2021-05-28,mylene baldon,dennis damian,UBP,96270.0,1,2021-05-27,erdinel cruz,mylene baldon,UBP,6000.0,1,1,2021-05-29,dennis damian,rogelio villangca,NonUBP,50000.0,1
922,2021-05-28,sweeny lumbang,maria gutierrez,UBP,4000.0,1,2021-05-27,maria gutierrez,sweeny lumbang,UBP,212000.0,1,1,2021-05-29,maria gutierrez,niel valderrama,NonUBP,2000.0,1


##### export level 3 data

In [40]:
level_3.to_csv('/home/cdsw/Data/sy_level_3.csv')

In [41]:
level_3_first.to_csv('/home/cdsw/Data/sy_level_3_first.csv')